In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.stats import randint 

# 导入数据集
file_path = '/Users/mxh/Downloads/data_副本.csv' # 替换为实际文件路径
data = pd.read_csv(file_path)

#随机抽取数据的 30%
data_sample = data.sample(frac=0.3, random_state=42)  # frac=0.3 表示抽取 30% 的数据

# 去除日期列
if 'earliesCreditLine' in data.columns:
    data = data.drop(columns=['earliesCreditLine'])

# 处理非数值数据
# 将所有非数值列转换为数值列（例如，将类别数据转换为数值编码）
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = pd.Categorical(data[column]).codes

print("\n缺失值检查：")
print(data.isnull().sum())

## 区分特征和标签
labels = data.iloc[:, 2]       # 选择除 'interestRate' 之外的所有列作为特征
features = data.drop(data.columns[2], axis=1)    # 选择 'interestRate' 作为标签

print("\n标签列名：")
print(data.columns[2])

# 检查标签的唯一值
print("\n标签的唯一值：")
print(labels.unique())


# 将标签转换为分类类型
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
# 分割数据集，80%作为训练集，20%作为测试集
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

# 标准化特征
scaler = StandardScaler()
X_train_standardized = scaler.fit_transform(X_train)
X_test_standardized = scaler.transform(X_test)

# 特征选择
model = ExtraTreesClassifier()
model.fit(X_train_standardized, y_train)

# 输出特征重要度
print("特征重要度：")
print(model.feature_importances_)

# 训练 ExtraTreesClassifier 并获取特征重要性
model = ExtraTreesClassifier()
model.fit(X_train_standardized, y_train)

# 保留 95% 的方差
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_standardized)
X_test_pca = pca.transform(X_test_standardized)

from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 定义 Random Forest 模型
rf = RandomForestClassifier(random_state=42)

# 定义超参数分布
param_distributions_rf = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(5, 50),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}

# 定义交叉验证策略
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# 定义随机搜索
random_search_rf = RandomizedSearchCV(estimator=rf, 
                                      param_distributions=param_distributions_rf, 
                                      n_iter=20, 
                                      cv=cv, 
                                      scoring='accuracy', 
                                      n_jobs=-1, 
                                      random_state=42, 
                                      verbose=2)

# 执行搜索
random_search_rf.fit(X_train_standardized, y_train)

# 输出最佳参数和最佳得分
print("Best parameters found for Random Forest: ", random_search_rf.best_params_)
print("Best accuracy score for Random Forest: ", random_search_rf.best_score_)

# 使用最佳参数训练模型并在测试集上评估
best_rf = random_search_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test_standardized)

# 计算模型的不同评分指标
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')

print(f"Random Forest Test set accuracy: {accuracy_rf}")
print(f"Random Forest Test set precision: {precision_rf}")
print(f"Random Forest Test set recall: {recall_rf}")
print(f"Random Forest Test set F1-score: {f1_rf}")



缺失值检查：
id                 0
loanAmnt           0
term               0
interestRate       0
installment        0
grade              0
subGrade           0
employmentTitle    0
annualIncome       0
isDefault          0
purpose            0
dti                0
ficoRangeLow       0
openAcc            0
revolBal           0
totalAcc           0
title              0
policyCode         0
dtype: int64

标签列名：
term

标签的唯一值：
[5 3]


KeyboardInterrupt: 

n_estimators: 在 100 到 500 之间随机选择树的数量。
max_depth: 随机选择树的最大深度，以控制模型的复杂性。
min_samples_split 和 min_samples_leaf: 控制分裂节点时的样本数，从而减少过拟合。
max_features: 控制每个分裂时可选择的特征数量，可以是 'auto'（默认，使用所有特征）、'sqrt'（平方根特征数）或 'log2'（对数特征数）。
bootstrap: 决定是否使用自助采样，True 为使用，False 为不使用。
通过这种随机搜索方法，可以找到性能最佳的参数组合，并且避免手动调整参数的繁琐过程。

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.stats import randint 
import joblib
# 导入数据集
file_path = '/Users/mxh/Downloads/data_副本.csv'# 替换为实际文件路径
data = pd.read_csv(file_path)

#随机抽取数据的 30%
data_sample = data.sample(frac=0.3, random_state=42)  # frac=0.3 表示抽取 30% 的数据

# 去除日期列
if 'earliesCreditLine' in data.columns:
    data = data.drop(columns=['earliesCreditLine'])

# 处理非数值数据
# 将所有非数值列转换为数值列（例如，将类别数据转换为数值编码）
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = pd.Categorical(data[column]).codes

print("\n缺失值检查：")
print(data.isnull().sum())

## 区分特征和标签
labels = data.iloc[:, 2]       # 选择除 'interestRate' 之外的所有列作为特征
features = data.drop(data.columns[2], axis=1)    # 选择 'interestRate' 作为标签

print("\n标签列名：")
print(data.columns[2])

# 检查标签的唯一值
print("\n标签的唯一值：")
print(labels.unique())


# 将标签转换为分类类型
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
# 分割数据集，80%作为训练集，20%作为测试集
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

# 标准化特征
scaler = StandardScaler()
X_train_standardized = scaler.fit_transform(X_train)
X_test_standardized = scaler.transform(X_test)

# 保存标准化器
joblib.dump(scaler, 'scaler.pkl')

# 特征选择
model = ExtraTreesClassifier()
model.fit(X_train_standardized, y_train)

# 输出特征重要度
print("特征重要度：")
print(model.feature_importances_)

# 训练 ExtraTreesClassifier 并获取特征重要性
model = ExtraTreesClassifier()
model.fit(X_train_standardized, y_train)

# 保留 95% 的方差
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_standardized)
X_test_pca = pca.transform(X_test_standardized)

from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 定义 Random Forest 模型
rf = RandomForestClassifier(random_state=42)

# 定义超参数分布
param_distributions_rf = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(5, 50),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}

# 定义交叉验证策略
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# 定义随机搜索
random_search_rf = RandomizedSearchCV(estimator=rf, 
                                      param_distributions=param_distributions_rf, 
                                      n_iter=50, 
                                      cv=cv, 
                                      scoring='accuracy', 
                                      n_jobs=-1, 
                                      random_state=42, 
                                      verbose=2)

# 执行搜索
random_search_rf.fit(X_train_standardized, y_train)

# 输出最佳参数和最佳得分
print("Best parameters found for Random Forest: ", random_search_rf.best_params_)
print("Best accuracy score for Random Forest: ", random_search_rf.best_score_)

# 使用最佳参数训练模型并在测试集上评估
best_rf = random_search_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test_standardized)

# 计算模型的不同评分指标
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')

print(f"Random Forest Test set accuracy: {accuracy_rf}")
print(f"Random Forest Test set precision: {precision_rf}")
print(f"Random Forest Test set recall: {recall_rf}")
print(f"Random Forest Test set F1-score: {f1_rf}")

# 保存最佳模型
joblib.dump(random_search_rf.best_estimator_, 'model.pkl')



缺失值检查：
id                 0
loanAmnt           0
term               0
interestRate       0
installment        0
grade              0
subGrade           0
employmentTitle    0
annualIncome       0
isDefault          0
purpose            0
dti                0
ficoRangeLow       0
openAcc            0
revolBal           0
totalAcc           0
title              0
policyCode         0
dtype: int64

标签列名：
term

标签的唯一值：
[5 3]
特征重要度：
[0.03102288 0.2201395  0.06769021 0.17426286 0.0792258  0.1014612
 0.03341879 0.0415413  0.01421234 0.01867488 0.03288253 0.05838004
 0.0306311  0.03987328 0.03595682 0.02062646 0.        ]
Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] END bootstrap=True, criterion=gini, max_depth=23, max_features=log2, min_samples_leaf=11, min_samples_split=12, n_estimators=187; total time= 4.4min
[CV] END bootstrap=True, criterion=gini, max_depth=23, max_features=log2, min_samples_leaf=11, min_samples_split=12, n_estimators=187; total time= 4.4min
[CV] EN

/Users/mxh/vscode/flask_ml_app/venv/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bootstrap=True, criterion=gini, max_depth=22, max_features=sqrt, min_samples_leaf=12, min_samples_split=3, n_estimators=301; total time= 8.9min
[CV] END bootstrap=False, criterion=entropy, max_depth=10, max_features=sqrt, min_samples_leaf=10, min_samples_split=5, n_estimators=409; total time=14.9min
[CV] END bootstrap=False, criterion=gini, max_depth=44, max_features=auto, min_samples_leaf=16, min_samples_split=14, n_estimators=437; total time=   0.1s
[CV] END bootstrap=False, criterion=gini, max_depth=44, max_features=auto, min_samples_leaf=16, min_samples_split=14, n_estimators=437; total time=   0.0s
[CV] END bootstrap=False, criterion=gini, max_depth=44, max_features=auto, min_samples_leaf=16, min_samples_split=14, n_estimators=437; total time=   0.0s
[CV] END bootstrap=True, criterion=gini, max_depth=22, max_features=sqrt, min_samples_leaf=12, min_samples_split=3, n_estimators=301; total time= 8.9min
[CV] END bootstrap=True, criterion=gini, max_depth=22, max_features=sqrt

/Users/mxh/vscode/flask_ml_app/venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
45 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/mxh/vscode/flask_ml_app/venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/mxh/vscode/flask_ml_app/venv/lib/python3.12/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/Users/mxh/vscode/flask_ml_app/venv/lib/python3.12/site-packages/sklearn/base.py", line 666, in _validate_param

Best parameters found for Random Forest:  {'bootstrap': False, 'criterion': 'gini', 'max_depth': 32, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 416}
Best accuracy score for Random Forest:  0.9342223239030423
Random Forest Test set accuracy: 0.9354718628973949
Random Forest Test set precision: 0.9345825748100489
Random Forest Test set recall: 0.9354718628973949
Random Forest Test set F1-score: 0.9347200065002335


['model.pkl']

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.stats import randint 
import joblib

# 导入数据集
file_path = '/Users/mxh/Downloads/data_副本.csv'  # 替换为实际文件路径
data = pd.read_csv(file_path)

# 随机抽取数据的 30%
data_sample = data.sample(frac=0.3, random_state=42)  # frac=0.3 表示抽取 30% 的数据

# 去除日期列
if 'earliesCreditLine' in data.columns:
    data = data.drop(columns=['earliesCreditLine'])

# 处理非数值数据
# 将所有非数值列转换为数值列（例如，将类别数据转换为数值编码）
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = pd.Categorical(data[column]).codes

print("\n缺失值检查：")
print(data.isnull().sum())

# 区分特征和标签
labels = data.iloc[:, 2]  # 选择第 2 列作为标签
features = data.drop(data.columns[2], axis=1)  # 选择除标签列外的所有列作为特征

print("\n标签列名：")
print(data.columns[2])

# 检查标签的唯一值
print("\n标签的唯一值：")
print(labels.unique())

# 将标签转换为分类类型
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# 分割数据集，80% 作为训练集，20% 作为测试集
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

# 标准化特征
scaler = StandardScaler()
X_train_standardized = scaler.fit_transform(X_train)
X_test_standardized = scaler.transform(X_test)

# 保存标准化器
joblib.dump(scaler, 'scaler.pkl')

# 特征选择
model = ExtraTreesClassifier()
model.fit(X_train_standardized, y_train)

# 输出特征重要度
print("特征重要度：")
print(model.feature_importances_)

# 保留 95% 的方差
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_standardized)
X_test_pca = pca.transform(X_test_standardized)

# 定义 Random Forest 模型
rf = RandomForestClassifier(random_state=42)

# 定义超参数分布
param_distributions_rf = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(5, 50),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['sqrt', 'log2', None],  # 替换 'auto' 为合法值
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}

# 定义交叉验证策略
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# 定义随机搜索
random_search_rf = RandomizedSearchCV(estimator=rf, 
                                      param_distributions=param_distributions_rf, 
                                      n_iter=50, 
                                      cv=cv, 
                                      scoring='accuracy', 
                                      n_jobs=-1, 
                                      random_state=42, 
                                      verbose=2)

# 执行搜索
random_search_rf.fit(X_train_standardized, y_train)

# 输出最佳参数和最佳得分
print("Best parameters found for Random Forest: ", random_search_rf.best_params_)
print("Best accuracy score for Random Forest: ", random_search_rf.best_score_)

# 使用最佳参数训练模型并在测试集上评估
best_rf = random_search_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test_standardized)

# 计算模型的不同评分指标
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')

print(f"Random Forest Test set accuracy: {accuracy_rf}")
print(f"Random Forest Test set precision: {precision_rf}")
print(f"Random Forest Test set recall: {recall_rf}")
print(f"Random Forest Test set F1-score: {f1_rf}")

# 保存最佳模型
joblib.dump(random_search_rf.best_estimator_, 'model.pkl')


缺失值检查：
id                 0
loanAmnt           0
term               0
interestRate       0
installment        0
grade              0
subGrade           0
employmentTitle    0
annualIncome       0
isDefault          0
purpose            0
dti                0
ficoRangeLow       0
openAcc            0
revolBal           0
totalAcc           0
title              0
policyCode         0
dtype: int64

标签列名：
term

标签的唯一值：
[5 3]
特征重要度：
[0.03163702 0.21636268 0.07225315 0.17216847 0.0798445  0.09653808
 0.033934   0.04234112 0.01353461 0.01891118 0.03323196 0.05988072
 0.03103465 0.04097552 0.03677276 0.0205796  0.        ]
Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] END bootstrap=True, criterion=gini, max_depth=23, max_features=None, min_samples_leaf=11, min_samples_split=12, n_estimators=187; total time=24.6min
[CV] END bootstrap=True, criterion=gini, max_depth=23, max_features=None, min_samples_leaf=11, min_samples_split=12, n_estimators=187; total time=24.8min
[CV] E

/Users/mxh/vscode/flask_ml_app/venv/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bootstrap=False, criterion=entropy, max_depth=25, max_features=sqrt, min_samples_leaf=12, min_samples_split=13, n_estimators=444; total time=27.9min
[CV] END bootstrap=False, criterion=entropy, max_depth=7, max_features=sqrt, min_samples_leaf=19, min_samples_split=8, n_estimators=120; total time= 4.7min
[CV] END bootstrap=True, criterion=entropy, max_depth=44, max_features=None, min_samples_leaf=2, min_samples_split=13, n_estimators=393; total time=59.0min
[CV] END bootstrap=True, criterion=entropy, max_depth=44, max_features=None, min_samples_leaf=2, min_samples_split=13, n_estimators=393; total time=60.2min
[CV] END bootstrap=False, criterion=entropy, max_depth=7, max_features=sqrt, min_samples_leaf=19, min_samples_split=8, n_estimators=120; total time= 4.7min
[CV] END bootstrap=True, criterion=gini, max_depth=46, max_features=None, min_samples_leaf=15, min_samples_split=13, n_estimators=154; total time=22.3min
[CV] END bootstrap=False, criterion=entropy, max_depth=7, max_fe

['model.pkl']

In [6]:
import os
print("当前工作目录：", os.getcwd())

当前工作目录： /Volumes/192.168.2.80/python练习/2week


In [7]:
import joblib

model = joblib.load('model.pkl')
print("Model feature names:", model.feature_names_in_)

AttributeError: 'RandomForestClassifier' object has no attribute 'feature_names_in_'

In [8]:
feature_names = X_train.columns.tolist()
print("Feature names:", feature_names)

Feature names: ['id', 'loanAmnt', 'interestRate', 'installment', 'grade', 'subGrade', 'employmentTitle', 'annualIncome', 'isDefault', 'purpose', 'dti', 'ficoRangeLow', 'openAcc', 'revolBal', 'totalAcc', 'title', 'policyCode']
